In [5]:
import cv2
import numpy as np
import os
import regex as re

# Video input
video_path = "C:\\Users\\chand\\Downloads\\SlowMoCric_2.mp4"
cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    print("Error: Could not open video file.")
    exit()

# Get FPS
original_fps = int(cap.get(cv2.CAP_PROP_FPS))
frame_time = 1 / original_fps

# Slow-motion factor (Increase to make video slower)
slow_motion_factor = 2  
new_fps = max(5, original_fps // slow_motion_factor)  # Reduce FPS for slower video

# Conversion factor (adjust based on calibration)
pixels_to_meters = 0.005  

# Create output directory
output_folder = 'Processed_Frames_2'
os.makedirs(output_folder, exist_ok=True)

prev_position = None
frame_idx = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # Ball color detection
    lower1, upper1 = np.array([0, 100, 0]), np.array([10, 255, 255])
    lower2, upper2 = np.array([160, 100, 20]), np.array([180, 255, 255])
    mask = cv2.inRange(hsv, lower1, upper1) + cv2.inRange(hsv, lower2, upper2)

    contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    detectedCircles = []

    for c in contours:
        area = cv2.contourArea(c)
        perimeter = cv2.arcLength(c, True)
        circularity = (4 * np.pi * area) / (perimeter**2) if perimeter != 0 else 0

        (x, y), radius = cv2.minEnclosingCircle(c)
        center = (int(x), int(y))
        radius = int(radius)

        if circularity > 0.2 and area > 35:
            detectedCircles.append([center, radius, area, circularity])

    if detectedCircles:
        largest_blob = max(detectedCircles, key=lambda x: x[2] * x[3])
        largest_center, largest_radius, _, _ = largest_blob
        x, y = largest_center

        if prev_position:
            # Calculate ball speed
            distance_pixels = np.linalg.norm(np.array(prev_position) - np.array((x, y)))
            distance_meters = distance_pixels * pixels_to_meters
            speed_mps = distance_meters / frame_time
            speed_kmh = speed_mps * 3.6

            if 10 < speed_kmh < 160:  # Filter out unrealistic speeds
                print(f"Ball Speed: {speed_kmh:.2f} km/h")
                speed_text = f"{speed_kmh:.2f} km/h"
                
                # Draw background rectangle for visibility
                text_size = cv2.getTextSize(speed_text, cv2.FONT_HERSHEY_SIMPLEX, 0.7, 2)[0]
                text_x, text_y = x + largest_radius + 10, y - 10  # Position near the ball
                cv2.rectangle(frame, (text_x - 5, text_y - text_size[1] - 5),
                              (text_x + text_size[0] + 5, text_y + 5), (0, 0, 0), -1)
                
                # Put text over rectangle
                cv2.putText(frame, speed_text, (text_x, text_y),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2, cv2.LINE_AA)

        prev_position = (x, y)

        # Draw ball tracking
        cv2.circle(frame, (x, y), largest_radius, (255, 0, 0), 2)

    # Save processed frame (Duplicate frames for slow motion)
    for _ in range(slow_motion_factor):  
        frame_filename = f"{output_folder}/frame_{frame_idx:04d}.png"
        cv2.imwrite(frame_filename, frame)
        frame_idx += 1

cap.release()

# Create final video from processed frames
output_video_path = 'Cric_Analysis_SlowMo_2.mp4'
frames = sorted(os.listdir(output_folder), key=lambda f: int(re.sub(r'\D', '', f)))

if frames:
    first_frame = cv2.imread(f"{output_folder}/{frames[0]}")
    if first_frame is not None:
        height, width, _ = first_frame.shape
        size = (width, height)

        out = cv2.VideoWriter(output_video_path, cv2.VideoWriter_fourcc(*'DIVX'), new_fps, size)

        for frame_name in frames:
            frame_path = f"{output_folder}/{frame_name}"
            frame = cv2.imread(frame_path)

            if frame is not None:
                out.write(frame)

        out.release()
        print("Slow-motion video saved as:", output_video_path)
    else:
        print("Error: No valid frames found to create the video.")
else:
    print("Error: No processed frames available.")


Ball Speed: 23.96 km/h
Ball Speed: 24.95 km/h
Ball Speed: 22.50 km/h
Ball Speed: 96.22 km/h
Ball Speed: 104.37 km/h
Ball Speed: 107.10 km/h
Ball Speed: 116.30 km/h
Ball Speed: 108.82 km/h
Ball Speed: 112.95 km/h
Ball Speed: 106.39 km/h
Ball Speed: 85.92 km/h
Ball Speed: 86.87 km/h
Ball Speed: 100.48 km/h
Ball Speed: 15.39 km/h
Ball Speed: 102.84 km/h
Ball Speed: 12.81 km/h
Ball Speed: 52.22 km/h
Ball Speed: 21.49 km/h
Ball Speed: 30.72 km/h
Ball Speed: 32.55 km/h
Ball Speed: 35.30 km/h
Ball Speed: 33.58 km/h
Ball Speed: 45.36 km/h
Ball Speed: 44.57 km/h
Ball Speed: 47.06 km/h
Ball Speed: 40.14 km/h
Ball Speed: 40.97 km/h
Ball Speed: 91.49 km/h
Ball Speed: 93.54 km/h
Ball Speed: 110.86 km/h
Ball Speed: 112.82 km/h
Slow-motion video saved as: Cric_Analysis_SlowMo_2.mp4
